# **Importing and Installing Dependencies**

In [ ]:
!pip install evaluate
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyli

In [ ]:
import pandas as pd
import numpy as np
import spacy
import pickle
import nltk
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize
from spacy.lang.en.stop_words import STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim.downloader
from gensim.models import Word2Vec
from scipy import spatial
from string import punctuation
from heapq import nlargest
import re
import networkx as nx
from evaluate import load
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/Text Mining Project/'
with open(path+'anime_summarized.pkl', 'rb') as f:
    anime_df = pickle.load(f)

In [ ]:
anime_df = anime_df[['Name', 'Genres','Synopsis','Producers','Studios', 'Image URL']]

In [ ]:
anime_df

,Name,Genres,Synopsis,Producers,Studios,Image URL
0,Cowboy Bebop,"Fantasy and Supernatural, Action and Adventure","Crime is timeless. By the year 2071, humanity ...",Bandai Visual,Sunrise,https://cdn.myanimelist.net/images/anime/4/196...
1,Cowboy Bebop: Tengoku no Tobira,"Fantasy and Supernatural, Action and Adventure","Another day, another bounty—such is the life o...","Sunrise, Bandai Visual",Bones,https://cdn.myanimelist.net/images/anime/1439/...
2,Trigun,"Fantasy and Supernatural, Action and Adventure","Vash the Stampede is the man with a $$60,000,0...",Victor Entertainment,Madhouse,https://cdn.myanimelist.net/images/anime/7/203...
3,Witch Hunter Robin,"Drama and Mistery, Action and Adventure, Fanta...",Robin Sena is a powerful craft user drafted in...,"Bandai Visual, Dentsu, Victor Entertainment, T...",Sunrise,https://cdn.myanimelist.net/images/anime/10/19...
4,Bouken Ou Beet,"Fantasy and Supernatural, Action and Adventure",It is the dark century and the people are suff...,"TV Tokyo, Dentsu",Toei Animation,https://cdn.myanimelist.net/images/anime/7/215...
...,...,...,...,...,...,...
9739,Mecha Mecha Karisuma,No genre identified,"This is the Charisma House. Today, too, the ch...",NaN,NaN,https://cdn.myanimelist.net/images/anime/1831/...
9740,Follow Your Fantasy,No genre identified,Witness stunning animation featuring Cygames a...,NaN,NaN,https://cdn.myanimelist.net/images/anime/1003/...
9741,Pon no Michi,"Action and Adventure, Comedy and Romance, Spor...","Nashiko Jippensha, a high school girl living i...","Bit grooove promotion, SUPA LOVE",OLM,https://cdn.myanimelist.net/images/anime/1796/...
9742,A Flash | Honkai: Star Rail,"Action and Adventure, Fantasy and Supernatural","Year 7380 Star Calendar, Jingliu, the mara-str...",NaN,NaN,https://cdn.myanimelist.net/images/anime/1547/...


# **Text Preprocessing for the Summarization**

In [ ]:
def clean_text(text):
  text = text.replace('\n',' ') #remove newlines
  text = re.sub(r"\([Ss]ource: [A-Za-z0-9]*\)", "", text) #remove the source of the synopsis
  text = " ".join(re.split("\s+", text, flags=re.UNICODE)) #remove multiple whitespaces
  return text

In [ ]:
anime_df['preprocessed_synopsis'] = anime_df['Synopsis'].apply(lambda x: clean_text(x))

In [ ]:
anime_df.reset_index(inplace = True, drop = True)

In [ ]:
anime_df

,Name,Genres,Synopsis,Producers,Studios,Image URL,preprocessed_synopsis
0,Cowboy Bebop,"[Fantasy and Supernatural, Action and Adventure]","Crime is timeless. By the year 2071, humanity ...",Bandai Visual,Sunrise,https://cdn.myanimelist.net/images/anime/4/196...,"Crime is timeless. By the year 2071, humanity ..."
1,Cowboy Bebop: Tengoku no Tobira,"[Fantasy and Supernatural, Action and Adventure]","Another day, another bounty—such is the life o...","Sunrise, Bandai Visual",Bones,https://cdn.myanimelist.net/images/anime/1439/...,"Another day, another bounty—such is the life o..."
2,Trigun,"[Fantasy and Supernatural, Action and Adventure]","Vash the Stampede is the man with a $$60,000,0...",Victor Entertainment,Madhouse,https://cdn.myanimelist.net/images/anime/7/203...,"Vash the Stampede is the man with a $$60,000,0..."
3,Witch Hunter Robin,"[Drama and Mistery, Action and Adventure, Fant...",Robin Sena is a powerful craft user drafted in...,"Bandai Visual, Dentsu, Victor Entertainment, T...",Sunrise,https://cdn.myanimelist.net/images/anime/10/19...,Robin Sena is a powerful craft user drafted in...
4,Bouken Ou Beet,"[Fantasy and Supernatural, Action and Adventure]",It is the dark century and the people are suff...,"TV Tokyo, Dentsu",Toei Animation,https://cdn.myanimelist.net/images/anime/7/215...,It is the dark century and the people are suff...
...,...,...,...,...,...,...,...
9739,Mecha Mecha Karisuma,NaN,"This is the Charisma House. Today, too, the ch...",NaN,NaN,https://cdn.myanimelist.net/images/anime/1831/...,"This is the Charisma House. Today, too, the ch..."
9740,Follow Your Fantasy,NaN,Witness stunning animation featuring Cygames a...,NaN,NaN,https://cdn.myanimelist.net/images/anime/1003/...,Witness stunning animation featuring Cygames a...
9741,Pon no Michi,NaN,"Nashiko Jippensha, a high school girl living i...","Bit grooove promotion, SUPA LOVE",OLM,https://cdn.myanimelist.net/images/anime/1796/...,"Nashiko Jippensha, a high school girl living i..."
9742,A Flash | Honkai: Star Rail,NaN,"Year 7380 Star Calendar, Jingliu, the mara-str...",NaN,NaN,https://cdn.myanimelist.net/images/anime/1547/...,"Year 7380 Star Calendar, Jingliu, the mara-str..."


# **Text Summarization**

## **Frequency Driven Approach - Word Probability**

Initially, we build a summary by going to construct scores for sentences based on the frequency of words within them.

In [ ]:
nlp = spacy.load('en_core_web_sm')
def summarize(text, per):
    doc= nlp(text)
    tokens=[token.text for token in doc]
    word_frequencies={} #Compute word frequencies
    for word in doc:
        if word.text.lower() not in list(STOP_WORDS):
            if word.text.lower() not in punctuation:
                if word.text not in word_frequencies.keys():
                    word_frequencies[word.text] = 1
                else:
                    word_frequencies[word.text] += 1
    max_frequency=max(word_frequencies.values()) #highest frequency of a word that is not a stopword or in puntuaction
    for word in word_frequencies.keys():
        word_frequencies[word]=word_frequencies[word]/max_frequency #compute relative frequencies
    sentence_tokens= [sent for sent in doc.sents] #divide the document in sentences
    sentence_scores = {}
    for sent in sentence_tokens:
        for word in sent:
            if word.text.lower() in word_frequencies.keys():
                if sent not in sentence_scores.keys():
                    sentence_scores[sent]=word_frequencies[word.text.lower()]
                else:
                    sentence_scores[sent]+=word_frequencies[word.text.lower()] #compute sentences scores based on the relative frequencies of the words within it
    select_length=int(len(sentence_tokens)*per) #select the number of sentences based on percentage
    summary=nlargest(select_length, sentence_scores,key=sentence_scores.get) #build the summary based on the scores of the sentences
    final_summary=[word.text for word in summary]
    summary=' '.join(final_summary)
    return summary

In [ ]:
anime_df['summarized_synopsis'] = anime_df['preprocessed_synopsis'].apply(lambda x: summarize(x, 0.5))

We evaluate the goodness of the summary using **BERTscore**.

It will give us 3 results:
* *Precision*
* *Recall*
* *F1 Score*

In [ ]:
bertscore = load("bertscore")
predictions = anime_df['summarized_synopsis'][0:500].tolist()
references = anime_df['Synopsis'][0:500].tolist()
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
print(f"The mean value for the precision is {np.array(results['precision']).mean()}")
print(f"The mean value for the recall is {np.array(results['recall']).mean()}")
print(f"The mean value for the f1 score is {np.array(results['f1']).mean()}")

The mean value for the precision is 0.9651909884214401
The mean value for the recall is 0.9018149521350861
The mean value for the f1 score is 0.9322634509801865


## **TextRank**

The second algorithm will be the **TextRank algorithm**.

Firstly, we download the vectorizer from *Gensim*.

In [ ]:
vectors = gensim.downloader.load('glove-wiki-gigaword-50')

[==================================================] 100.0% 66.0/66.0MB downloaded


Defining a function which normalize the documents.

In [ ]:
def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in STOP_WORDS]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

We define a function which allow us to create the embeddings for the sentences based on the vectors of each word within the sentence.

In [ ]:
def create_matrix(sentences):
  matrix = []
  for sent in sentences:
    word_count = 0 #keep the count of the word in the phrase
    sent_vector = np.zeros(50) #define the vector of the sentence
    for word in sent: #iterate on the words in the sentence
      if word in vectors.index_to_key:
        sent_vector += vectors.get_vector(word) #sum the vectors of the words
        word_count += 1 #update the count of words in the sentence
    sent_vector = sent_vector/word_count #normalize the sentence vector with the number of words
    matrix.append(sent_vector) #append the vector of the sentence to the document embedding
  return matrix

We define the function that builds the summaries.

In [ ]:
def textrank_word2vec(text, per):
  try:
    sentences = sent_tokenize(text) #tokenize the sentences
    norm_sentences = normalize_corpus(sentences) #normalize the sentences

    matrix = np.array(create_matrix(norm_sentences)) #build the embedding for the document
    similarity_matrix = np.matmul(matrix, matrix.T) #build the similarity matrix

    similarity_graph = nx.from_numpy_array(similarity_matrix) #build the graph based on the similarity matrix
    scores = nx.pagerank(similarity_graph, alpha = 0.9) #apply the pagerank algorithm to the graph
    ranked_sentences = sorted(((score, index) for index, score
                                            in scores.items()), reverse=True) #sort the sentences according to the values from the algorithm
    num_sentences = int(len(sentences)**per) #select the length of the summary
    top_sentence_indices = [ranked_sentences[index][1] #consider the top sentences
                        for index in range(num_sentences)]
    top_sentence_indices.sort()
    return ' '.join(np.array(sentences)[top_sentence_indices]) #return the summary

  except nx.PowerIterationFailedConvergence:
    return text #in case of non-convergence of the pagerank algorithm, returns the original text


In [ ]:
anime_df['textrank_summary'] = anime_df['preprocessed_synopsis'].apply(lambda x: textrank_word2vec(x, per = 0.5))

In [ ]:
anime_df

,Name,Genres,Synopsis,Producers,Studios,Image URL,preprocessed_synopsis,summarized_synopsis,textrank_summary
0,Cowboy Bebop,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",Bandai Visual,Sunrise,https://cdn.myanimelist.net/images/anime/4/196...,"Crime is timeless. By the year 2071, humanity ...","These new societies are plagued by murder, dru...",Crime is timeless. These new societies are pla...
1,Cowboy Bebop: Tengoku no Tobira,"Action, Sci-Fi","Another day, another bounty—such is the life o...","Sunrise, Bandai Visual",Bones,https://cdn.myanimelist.net/images/anime/1439/...,"Another day, another bounty—such is the life o...","Through their individual investigations, they ...","Another day, another bounty—such is the life o..."
2,Trigun,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",Victor Entertainment,Madhouse,https://cdn.myanimelist.net/images/anime/7/203...,"Vash the Stampede is the man with a $$60,000,0...",With his crazy doughnut obsession and buffooni...,The reason: he's a merciless villain who lays ...
3,Witch Hunter Robin,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,"Bandai Visual, Dentsu, Victor Entertainment, T...",Sunrise,https://cdn.myanimelist.net/images/anime/10/19...,Robin Sena is a powerful craft user drafted in...,Robin Sena is a powerful craft user drafted in...,But the truth about the Witches and herself wi...
4,Bouken Ou Beet,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,"TV Tokyo, Dentsu",Toei Animation,https://cdn.myanimelist.net/images/anime/7/215...,It is the dark century and the people are suff...,It is the dark century and the people are suff...,The Vandel Busters are a group of people who h...
...,...,...,...,...,...,...,...,...,...
9739,The Demon Queen Has a Death Wish,"Fantasy, Romance",I wanted to die.. until I met him.\nAfter brea...,NaN,NaN,https://cdn.myanimelist.net/images/anime/1527/...,I wanted to die.. until I met him. After break...,After breaking a treaty and waging war against...,I wanted to die.. until I met him. But the Dem...
9740,No More Princes,"Fantasy, Romance",html.serialization_informationFIN.\nジャンル\nFant...,NaN,NaN,https://cdn.myanimelist.net/img/sp/icon/apple-...,html.serialization_informationFIN. ジャンル Fant. ...,"This is the legendary tale of valiant princes,...",ジャンル Fant. Romance ArtistMr. Please take care ...
9741,The Forgotten Princess Just Wants Peace,"Fantasy, Romance",I'm the daughter of a Duke?!\nWhile locked up ...,NaN,NaN,https://cdn.myanimelist.net/images/anime/1887/...,I'm the daughter of a Duke?! While locked up i...,"While locked up in a prison cell, Eluana Vita ...","I'm the daughter of a Duke?! However, she's in..."
9742,Beauty and the Brawn,"Boys Love, Comedy",Haeun is a nobleman who falls hard for Dullseh...,NaN,NaN,https://cdn.myanimelist.net/images/anime/1269/...,Haeun is a nobleman who falls hard for Dullseh...,Silly hijinks ensue as Haeun will do whatever ...,Haeun is a nobleman who falls hard for Dullseh...


## **Evaluation**

In [ ]:
predictions = anime_df['textrank_summary'][0:500].tolist()
references = anime_df['Synopsis'][0:500].tolist()
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")

print(f"The mean value for the precision is {np.array(results['precision']).mean()}")
print(f"The mean value for the recall is {np.array(results['recall']).mean()}")
print(f"The mean value for the f1 score is {np.array(results['f1']).mean()}")

The mean value for the precision is 0.9616506346464158
The mean value for the recall is 0.8049813454151153
The mean value for the f1 score is 0.875792985200882


# **Hybrid Approach**

We define a new approach called **Hybrid Approach**, a mix between the first and the second approach previously defined. It is based on the following steps:
1. Try the **TextRank algorithm**
2. If the algorithm converges, returns the summary made with the **TextRank**
3. If it does not converge, returns the summary made with the first method

In [ ]:
def hybrid_summary(text, per):
  try:
    sentences = sent_tokenize(text)

    if len(sentences) <= 2:
      return text

    else:
      norm_sentences = normalize_corpus(sentences)

      matrix = np.array(create_matrix(norm_sentences))
      similarity_matrix = np.matmul(matrix, matrix.T)

      similarity_graph = nx.from_numpy_array(similarity_matrix)
      scores = nx.pagerank(similarity_graph, alpha = 0.9)
      ranked_sentences = sorted(((score, index) for index, score
                                            in scores.items()), reverse=True)
      num_sentences = int(len(sentences)**per)
      top_sentence_indices = [ranked_sentences[index][1]
                        for index in range(num_sentences)]
      top_sentence_indices.sort()
      return ' '.join(np.array(sentences)[top_sentence_indices])

  except nx.PowerIterationFailedConvergence:
    return summarize(text, per = per)

In [ ]:
anime_df['hybrid_summary'] = anime_df['preprocessed_synopsis'].apply(lambda x: hybrid_summary(x, per = 0.5))

## **Evaluation**

In [ ]:
predictions = anime_df['hybrid_summary'][0:500].tolist()
references = anime_df['Synopsis'][0:500].tolist()
results = bertscore.compute(predictions=predictions, references=references, model_type="distilbert-base-uncased")

print(f"The mean value for the precision is {np.array(results['precision']).mean()}")
print(f"The mean value for the recall is {np.array(results['recall']).mean()}")
print(f"The mean value for the f1 score is {np.array(results['f1']).mean()}")

The mean value for the precision is 0.9619603939056397
The mean value for the recall is 0.8078242839574814
The mean value for the f1 score is 0.8775051863193511


The first algorithm is the faster and the one which performs better according to the **BERTscore** evaluation, so we will consider the summaries created with the first one.

In [ ]:
anime_df = anime_df[['Name','Genres','Synopsis','Producers', 'Studios', 'summarized_synopsis','Image URL']]

# **Saving Dataframe**

In [ ]:
with open(path + 'anime_summarized.pkl', 'wb') as f:
  anime_df.to_pickle(f)